In [3]:
import praw

# initialize Reddit Object
reddit = praw.Reddit(
    client_id="XXX",
    client_secret="XXX",
    password="XX",
    user_agent="XXX",
    username="XXX",
)

In [4]:
import datetime
import pandas as pd

# define subreddit 

subreddits = ['datascience', 'MachineLearning', 'deeplearning', 'chatGPT','programming']

# create a temporary df to store results in
posts_today = pd.DataFrame(columns = ['date', 'subreddit', 'post_title', 'score'])

for subreddit in subreddits:
    subreddit = reddit.subreddit(subreddit)

    # iterate over submissions in the subreddit
    for submission in subreddit.new(limit=1000): 
        creation_time = datetime.datetime.fromtimestamp(submission.created_utc)

        if len(submission.comments) ==1:
            if creation_time.date() == datetime.datetime.today().date():
                posts_today = pd.concat([posts_today, pd.DataFrame({'date': [datetime.datetime.today().date()],
                                                                'subreddit': [subreddit],
                                                                'post_title': [submission.title],
                                                                'post_body': [submission.selftext],
                                                                'score': [submission.score]})])

        if len(submission.comments) ==2:
            if creation_time.date() == datetime.datetime.today().date():
                posts_today = pd.concat([posts_today, pd.DataFrame({'date': [datetime.datetime.today().date()],
                                                                'subreddit': [subreddit],
                                                                'post_title': [submission.title],
                                                                'post_body': [submission.selftext],
                                                                'score': [submission.score],
                                                                'top_comment_one':[submission.comments[1].body]})])

        if len(submission.comments) ==3:
            if creation_time.date() == datetime.datetime.today().date():
                posts_today = pd.concat([posts_today, pd.DataFrame({'date': [datetime.datetime.today().date()],
                                                                'subreddit': [subreddit],
                                                                'post_title': [submission.title],
                                                                'post_body': [submission.selftext],
                                                                'score': [submission.score],
                                                                'top_comment_one':[submission.comments[1].body],
                                                                'top_comment_two':[submission.comments[2].body]})])
                                                
        if len(submission.comments) >=4:
            if creation_time.date() == datetime.datetime.today().date():
                posts_today = pd.concat([posts_today, pd.DataFrame({'date': [datetime.datetime.today().date()],
                                                                'subreddit': [subreddit],
                                                                'post_title': [submission.title],
                                                                'post_body': [submission.selftext],
                                                                'score': [submission.score],
                                                                'top_comment_one':[submission.comments[1].body],
                                                                'top_comment_two':[submission.comments[2].body],
                                                                'top_comment_three':[submission.comments[3].body]})])

In [ ]:
from sqlalchemy import create_engine

posts_today.fillna(0, inplace = True)

# slicuje tytuly bo byly za dlugie

posts_today['post_title'] = posts_today['post_title'].str.slice(0, 200)

# same with post body
posts_today['post_body'] = posts_today['post_body'].str.slice(0, 10000)

# Connect to the database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="root", 
                               pw="mabestian9000",
                               host="34.68.170.215",
                               db="reddit_scraping"))

posts_today.to_sql("reddit_posts", engine, if_exists="append", index=False)                        

In [ ]:
# tutaj zmienialem typ kodowania naszej bazy danych zeby miala wiekszy zakres znakow

'''import mysql.connector


conn = mysql.connector.connect(
  host="34.68.170.215",
  user="root",
  password="mabestian9000",
  database="reddit_scraping"
)

cursor = conn.cursor()
cursor.execute("ALTER DATABASE reddit_scraping CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")
cursor.execute("ALTER TABLE reddit_posts CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci")
conn.commit()

# tutaj z kolei zwiekszylem wielkosc post_title bo niektore tytuly byly dluzsze

cursor = conn.cursor()

cursor.execute("ALTER TABLE reddit_posts MODIFY post_body VARCHAR(16000)")

conn.commit()'''